加载预处理好的dataframe文件，用的是movielen里1M的数据集。

In [1]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', 
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['user_id'].drop_duplicates().max()
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Reading ratings file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', 
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading ratings file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', 
                     usecols=['movie_id', 'title', 'genres'])

RNG_SEED参考代码里没有定义，因为是SEED，所以给个随机值就好。

In [2]:
# *cannot find RNG_SEED
RNG_SEED = 123

# Create training set
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print 'Users:', Users, ', shape =', Users.shape

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print 'Movies:', Movies, ', shape =', Movies.shape

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print 'Ratings:', Ratings, ', shape =', Ratings.shape

Users: [2232 4273 2497 ...  194 1675 4610] , shape = (1000209,)
Movies: [ 439  586  453 ... 2513 2565 1887] , shape = (1000209,)
Ratings: [4 5 3 ... 3 3 1] , shape = (1000209,)


CFModel为参考代码里作者自己写的生成Embedding矩阵的模块，原文见，

# Deep Learning Model
The idea of using deep learning is similar to that of Model-Based Matrix Factorization. In matrix factorizaion, we decompose our original sparse matrix into product of 2 low rank orthogonal matrices. For deep learning implementation, we don’t need them to be orthogonal, we want our model to learn the values of embedding matrix itself. The user latent features and movie latent features are looked up from the embedding matrices for specific movie-user combination. These are the input values for further linear and non-linear layers. We can pass this input to multiple relu, linear or sigmoid layers and learn the corresponding weights by any optimization algorithm (Adam, SGD, etc.).

### Build the Model
I created a sparse matrix factoring algorithm in Keras for my model in [CFModel.py](https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py). Here are the main components:

* A left embedding layer that creates a Users by Latent Factors matrix.
* A right embedding layer that creates a Movies by Latent Factors matrix.
* When the input to these layers are (i) a user id and (ii) a movie id, they'll return the latent factor vectors for the user and the movie, respectively.
* A merge layer that takes the dot product of these two latent vectors to return the predicted rating.

This code is based on the approach outlined in [Alkahest](http://www.fenris.org/)'s blog post [Collaborative Filtering in Keras](http://www.fenris.org/2016/03/07/index-html).

![embedding-layers](images/embedding-layers.png)



In [3]:
# Import Keras libraries
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
# Import CF Model Architecture
from CFModel import CFModel

Using TensorFlow backend.


K_FACTORS在此处的用途暂时不明
TEST_USER，随机抽取的一个测试用户的ID

In [4]:
# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 2000 # A random test user (user_id = 2000)

添加神经网络以及指定loss和optimizer。
*我理解届时要测试的就是不同的loss和optimizer的组合的效果*
此外参考代码用的Keras版本较旧，用最新版本已无法执行（CFModel.py中涉及已经被淘汰keras.layers.Merge)。如果需要改用最新版本，需要花点时间适配，当前通过conda uninstall keras && pip install keras==2.0来让他先跑起来

In [5]:
# Define model
model = CFModel(max_userid, max_movieid, K_FACTORS)
# Compile the model using MSE as the loss function and the AdaMax learning algorithm
model.compile(loss='mse', optimizer='adamax')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/ubuntu/anaconda3/envs/keras_p27/lib/python2.7/site-packages/keras/legacy/layers.py:66: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  warnings.warn('The `Merge` layer is deprecated '


此步骤开始训练模型，测量的指标是val_loss，将那么多epoch跑下来后，val_loss最低的模型落地为weights.h5。
patiences = 2的意思是，当发现观察的指标开始恶化2回后，就不继续训练了。
数据集中的10%将作为验证数据集，用来测量val_loss。

In [6]:
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint('weights.h5', save_best_only=True)]

# Use 30 epochs, 90% training data, 10% validation data 
history = model.fit([Users, Movies], Ratings, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

Train on 900188 samples, validate on 100021 samples
Epoch 1/30
91s - loss: 8.1846 - val_loss: 2.2510
Epoch 2/30
89s - loss: 1.4802 - val_loss: 1.1229
Epoch 3/30
88s - loss: 0.9976 - val_loss: 0.9327
Epoch 4/30
88s - loss: 0.8841 - val_loss: 0.8682
Epoch 5/30
88s - loss: 0.8342 - val_loss: 0.8373
Epoch 6/30
88s - loss: 0.8044 - val_loss: 0.8184
Epoch 7/30
88s - loss: 0.7818 - val_loss: 0.8040
Epoch 8/30
87s - loss: 0.7620 - val_loss: 0.7930
Epoch 9/30
87s - loss: 0.7433 - val_loss: 0.7837
Epoch 10/30
87s - loss: 0.7244 - val_loss: 0.7737
Epoch 11/30
87s - loss: 0.7051 - val_loss: 0.7663
Epoch 12/30
87s - loss: 0.6853 - val_loss: 0.7591
Epoch 13/30
87s - loss: 0.6650 - val_loss: 0.7551
Epoch 14/30
87s - loss: 0.6447 - val_loss: 0.7493
Epoch 15/30
87s - loss: 0.6240 - val_loss: 0.7473
Epoch 16/30
87s - loss: 0.6034 - val_loss: 0.7458
Epoch 17/30
87s - loss: 0.5827 - val_loss: 0.7458
Epoch 18/30
87s - loss: 0.5624 - val_loss: 0.7479
Epoch 19/30
87s - loss: 0.5425 - val_loss: 0.7506


可以看到在epoch到9/30的时候达到最低（每次执行的结果会不一样，推测是因为随机切分的验证数据不一样），后续开始逐步上升。说明已经开始出现过拟合，迭代的次数？到此为止。
下面的代码从history中找出上面步骤的最低的val_loss，开平方根算出RMSE。

In [ ]:
# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

Minimum RMSE at epoch 16 = 0.8636


我理解之后的测试做法是，
1. 用不同的loss/optimizer组合得出不同的RMSE
2. 记录下在同样数据情况下，不同组合的不同RMSE，以及epoch的值